# Atividade Prática II - Treinamento e Validação de Modelos de RL

**Aluno:** Recigio Poffo

**Disciplina:** Reinforcement Learning - Turma I

**Data:** 03/07/2021



Neste trabalho vamos aplicar `Gym`, `Stable-Baselines3` e `RL Baselines Zoo` para lidar com o treinamento e validação de problemas de aprendizado por reforço. Sua tarefa é:

1. Selecionar um cenário da biblioteca `Gym` de sua preferência, desde que este cenário também seja contemplado pelos modelos disponibilizados na `rl baselines zoo`;
2. Selecionar três algoritmos das biblioteca `Stable-baselines3` para resolver esse problema. Pesquise na documentação da biblioteca quais são os algoritmos mais adequados para o ambiente escolhido e justifique a sua escolha. 
3. Realize o treinamento de cada um dos três modelos ---você pode ajustar os parâmetros do modelos, se achar necessário--- e salve os modelos em disco.
4. De posse dos modelos treinados e salvos, carregue-os e avalie-os por 10 episódios. Apresente os resultados médios e gere a curva de recompensa acumulada disponibilizada pelo `TensorBoard`.
5. Compare os resultados dos modelos treinados com os resultados obtidos por modelo(s) existentes no `RL Baselines Zoo` para o cenário escolhido.
6. Gere um vídeo do melhor modelo que você treinou e do modelo escolhido na `RL Baselines Zoo`. Verifique a documentação de cada biblioteca sobre a criação do vídeo e visualização em Notebooks.



* **Data de entrega:** 16/07/2021
* **Local de envio:** AVA.
* **Tipo de documento:** Notebook (`.ipynb`).



### Testando ambiente base 

In [ ]:
#importa a biblioteca gym
import gym
import time


scenario = 'BeamRiderNoFrameskip-v4'
env = gym.make(scenario)

observation = env.reset()
#define a quantidade de passos de tempo
for _ in range(100):
  #desenha a visualização do ambiente  
  env.render()
  #toma uma ação de forma aleatória
  action = env.action_space.sample()
  #observa o ambiente com a base na ação executada
  observation, reward, done, info = env.step(action)
  #adiciona um atraso na execução para melhorar a visualização
  time.sleep(0.03)
  #verifica se o agente completou a tarefa
  if done:
    #reinicia a simulação
    observation = env.reset()
#fecha a visualização
env.close()

In [20]:
from IPython.display import Image

### Escolhendo os modelos

#### Foi escolhido o ambiente BeamRiderNoFrameskip-v4

#### Foram escolhidos os modelos A2C, PPO,DQN. Foram utilizados estes por que eles funcionam razoavelmente diferentes pelo que eu pesquisei na documentação do https://stable-baselines3.readthedocs.io/en/master/. 


#### A2C: https://openai.com/blog/baselines-acktr-a2c/
#### PPO: https://openai.com/blog/openai-baselines-ppo/
#### DQN: https://www.nature.com/articles/nature14236

#### O A2C é baseado no A3C e faz uso de multiplos trabalhadores tendêndeciando ao gradiente natural. O PPO, que é um algoritmo que tenta resolver problema scom gradientes descendentes, que podem demorar muito ou serem dificeis de achar um "tunning" ideal. Por fim, o DQN, que tem como base o Q-Learning, mais especificamente a versão "Neural Fitted Q Iteration"

In [ ]:
#tensorboard --logdir /tmp/stable-baselines/

### Considerações Iniciais

In [ ]:
#Para o modelo gerar a saida de dados de eval no tensor flow, a unica forma que encontrei foi criar dois ambientes,
#um de treinamento e outro e avaliação. Além disso, precisei setar a frequência do eval e o numero de episodios.
#coloquei apenas 1 avaliação por que acredito que o simulador ficava "parado", como explico melhor mais adiante e 
#demorava muito para continuaro treinamento. Treinamento feito inicialmente no 2e6, com mesmo numero para comparar no ZOO

### A2C

In [1]:
import gym
import time
from stable_baselines3 import A2C

#carrega o ambiente
env_a2c = gym.make('BeamRiderNoFrameskip-v4')
eval_env_a2c = gym.make('BeamRiderNoFrameskip-v4')

#instancia o algoritmo de aprendizagem
model = A2C('MlpPolicy', env_a2c, verbose=1,create_eval_env=True,tensorboard_log="/tmp/stable-baselines/BeamRiderNoFrameskip-v4/custon_a2c")

#treina o algoritmo por 2e5 passos de tempo
model.learn(total_timesteps=int(2e6),eval_env=eval_env_a2c,eval_freq=200000, n_eval_episodes=1)


#### Visualização do primeiro resultado

In [ ]:
import time

for _ in range(2):
    obs = env_a2c.reset()
    for i in range(1000):
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, done, info = env_a2c.step(action)
        time.sleep(0.003)
        env_a2c.render()
        
env_a2c.close()

c:\xampp\htdocs\posfurb\aprendizadodemaquina\python\.env\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Não é possível alterar o modo de thread depois de o mesmo estar definido
  warnings.warn(str(err))


In [6]:
env_a2c.close()

#### Salvamento, carregamento e avaliação do modelo

In [2]:
# Salva o agente
model.save("modelos/a2c/recigio_beamrider2M")

In [3]:
# deleta o agente/modelo
del model  

In [2]:
# carrega um modelo já treinado
model = A2C.load("modelos/a2c/recigio_beamrider2M", env=env_a2c)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [ ]:
# importando biblioteca de avaliação e avaliando por 10 episodios
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10, render=True,)

In [13]:
print(mean_reward)
print(std_reward)

660.0
0.0


### PPO

In [5]:
import gym
import time
from stable_baselines3 import PPO

#carrega o ambiente
env_ppo = gym.make('BeamRiderNoFrameskip-v4')
eval_env_ppo = gym.make('BeamRiderNoFrameskip-v4')

#instancia o algoritmo de aprendizagem
model2 = PPO('MlpPolicy', env_ppo, verbose=1,create_eval_env=True,tensorboard_log="/tmp/stable-baselines/BeamRiderNoFrameskip-v4/custon_ppo")

#treina o algoritmo por 2e5 passos de tempo
model2.learn(total_timesteps=int(2e6),eval_env=eval_env_ppo,eval_freq=200000, n_eval_episodes=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to /tmp/stable-baselines/BeamRiderNoFrameskip-v4/custon_ppo\PPO_1
-----------------------------
| time/              |      |
|    fps             | 397  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 191          |
|    iterations           | 2            |
|    time_elapsed         | 21           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0107437195 |
|    clip_fraction        | 0.0756       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.1

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.55e+03    |
|    ep_rew_mean          | 381         |
| time/                   |             |
|    fps                  | 134         |
|    iterations           | 11          |
|    time_elapsed         | 167         |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.010906251 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.1        |
|    explained_variance   | -0.0306     |
|    learning_rate        | 0.0003      |
|    loss                 | 29.4        |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.018      |
|    value_loss           | 46          |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.03e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.14e+03    |
|    ep_rew_mean          | 455         |
| time/                   |             |
|    fps                  | 127         |
|    iterations           | 21          |
|    time_elapsed         | 338         |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.014205655 |
|    clip_fraction        | 0.151       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.06       |
|    explained_variance   | -0.0521     |
|    learning_rate        | 0.0003      |
|    loss                 | 20          |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.0163     |
|    value_loss           | 22.4        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.23e+

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.27e+03     |
|    ep_rew_mean          | 427          |
| time/                   |              |
|    fps                  | 126          |
|    iterations           | 31           |
|    time_elapsed         | 499          |
|    total_timesteps      | 63488        |
| train/                  |              |
|    approx_kl            | 0.0108986115 |
|    clip_fraction        | 0.086        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.95        |
|    explained_variance   | -0.0229      |
|    learning_rate        | 0.0003       |
|    loss                 | 18.3         |
|    n_updates            | 300          |
|    policy_gradient_loss | -0.00847     |
|    value_loss           | 36.1         |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.59e+03    |
|    ep_rew_mean          | 445         |
| time/                   |             |
|    fps                  | 126         |
|    iterations           | 41          |
|    time_elapsed         | 663         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.008661712 |
|    clip_fraction        | 0.0611      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.78       |
|    explained_variance   | -0.0284     |
|    learning_rate        | 0.0003      |
|    loss                 | 10          |
|    n_updates            | 400         |
|    policy_gradient_loss | -0.00969    |
|    value_loss           | 15.7        |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 6.49e+03

...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 7.73e+03   |
|    ep_rew_mean          | 475        |
| time/                   |            |
|    fps                  | 104        |
|    iterations           | 932        |
|    time_elapsed         | 18270      |
|    total_timesteps      | 1908736    |
| train/                  |            |
|    approx_kl            | 0.04747235 |
|    clip_fraction        | 0.116      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.327     |
|    explained_variance   | -3.46      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0109    |
|    n_updates            | 9310       |
|    policy_gradient_loss | -0.00859   |
|    value_loss           | 0.116      |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.73e+03    |
|    ep_rew_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.49e+03    |
|    ep_rew_mean          | 465         |
| time/                   |             |
|    fps                  | 104         |
|    iterations           | 942         |
|    time_elapsed         | 18416       |
|    total_timesteps      | 1929216     |
| train/                  |             |
|    approx_kl            | 0.004127317 |
|    clip_fraction        | 0.0617      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.316      |
|    explained_variance   | -0.00274    |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0464      |
|    n_updates            | 9410        |
|    policy_gradient_loss | -0.00254    |
|    value_loss           | 7.9         |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.49

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.28e+03     |
|    ep_rew_mean          | 461          |
| time/                   |              |
|    fps                  | 105          |
|    iterations           | 952          |
|    time_elapsed         | 18567        |
|    total_timesteps      | 1949696      |
| train/                  |              |
|    approx_kl            | 0.0025619068 |
|    clip_fraction        | 0.039        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.218       |
|    explained_variance   | -0.0012      |
|    learning_rate        | 0.0003       |
|    loss                 | 6.91         |
|    n_updates            | 9510         |
|    policy_gradient_loss | -0.00114     |
|    value_loss           | 24.6         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.17e+03    |
|    ep_rew_mean          | 457         |
| time/                   |             |
|    fps                  | 105         |
|    iterations           | 962         |
|    time_elapsed         | 18714       |
|    total_timesteps      | 1970176     |
| train/                  |             |
|    approx_kl            | 0.004552854 |
|    clip_fraction        | 0.0321      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.265      |
|    explained_variance   | 0.00515     |
|    learning_rate        | 0.0003      |
|    loss                 | 0.552       |
|    n_updates            | 9610        |
|    policy_gradient_loss | -0.00109    |
|    value_loss           | 8.04        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.17

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.9e+03     |
|    ep_rew_mean          | 449         |
| time/                   |             |
|    fps                  | 105         |
|    iterations           | 972         |
|    time_elapsed         | 18862       |
|    total_timesteps      | 1990656     |
| train/                  |             |
|    approx_kl            | 0.002558792 |
|    clip_fraction        | 0.0336      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.23       |
|    explained_variance   | 0.0132      |
|    learning_rate        | 0.0003      |
|    loss                 | 15.1        |
|    n_updates            | 9710        |
|    policy_gradient_loss | -0.00222    |
|    value_loss           | 43.4        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.9e

#### Visualização do primeiro resultado

In [16]:
import time

for _ in range(2):
    obs = env_ppo.reset()
    for i in range(1000):
        action, _states = model2.predict(obs, deterministic=True)
        obs, rewards, done, info = env_ppo.step(action)
        time.sleep(0.003)
        env_ppo.render()
        
env_ppo.close()

In [ ]:
env_ppo.close()

#### Salvamento, carregamento e avaliação do modelo

In [6]:
# Salva o agente
model2.save("modelos/ppo/recigio_beamrider2M")

c:\xampp\htdocs\posfurb\aprendizadodemaquina\python\.env\lib\site-packages\stable_baselines3\common\save_util.py:276: UserWarning: Path 'modelos\ppo' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [7]:
# deleta o agente/modelo
del model2  

In [9]:
# carrega um modelo já treinado
model2 = PPO.load("modelos/ppo/recigio_beamrider2M", env=env_ppo)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [ ]:
# importando biblioteca de avaliação e avaliando por 10 episodios
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model2, model2.get_env(), n_eval_episodes=10, render=True)

c:\xampp\htdocs\posfurb\aprendizadodemaquina\python\.env\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Não é possível alterar o modo de thread depois de o mesmo estar definido
  warnings.warn(str(err))


In [22]:
print(mean_reward)
print(std_reward)

0.0
0.0


### DQN

In [1]:
import gym
import time
from stable_baselines3 import DQN

#carrega o ambiente
env_dqn = gym.make('BeamRiderNoFrameskip-v4')
eval_env_dqn = gym.make('BeamRiderNoFrameskip-v4')

#Para o DQN, para ele conseguir rodar, com a memória que tenho na máquina, tive que setar o batch e buffer_size menor.
#instancia o algoritmo de aprendizagem
model3 = DQN('MlpPolicy', env_dqn, verbose=1,batch_size=128, buffer_size=10000,create_eval_env=True,tensorboard_log="/tmp/stable-baselines/BeamRiderNoFrameskip-v4/custon_dqn")

#treina o algoritmo por 2e5 passos de tempo
model3.learn(total_timesteps=int(2e6),eval_env=eval_env_dqn,eval_freq=200000, n_eval_episodes=1)

#### Visualização do primeiro resultado

In [ ]:
import time

for _ in range(2):
    obs = env_dqn.reset()
    for i in range(1000):
        action, _states = model3.predict(obs, deterministic=True)
        obs, rewards, done, info = env_dqn.step(action)
        time.sleep(0.003)
        env_dqn.render()
env.close()

#### Salvamento, carregamento e avaliação do modelo

In [4]:
# Salva o agente
model3.save("modelos/dqn/recigio_beamrider2M")

c:\xampp\htdocs\posfurb\aprendizadodemaquina\python\.env\lib\site-packages\stable_baselines3\common\save_util.py:276: UserWarning: Path 'modelos\dqn' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [5]:
# deleta o agente/modelo
del model3 

In [2]:
# carrega um modelo já treinado
model3 = DQN.load("modelos/dqn/recigio_beamrider_2M", env=env_dqn)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [3]:
# importando biblioteca de avaliação e avaliando por 10 episodios
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model3, model3.get_env(), n_eval_episodes=10, render=True)

c:\xampp\htdocs\posfurb\aprendizadodemaquina\python\.env\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Não é possível alterar o modo de thread depois de o mesmo estar definido
  warnings.warn(str(err))


In [4]:
print(mean_reward)
print(std_reward)

44.0
0.0


#### Análise do problema

In [10]:
# Observei que acontece uma curva no treinamento, onde ele melhora até 800k timesteps, ou um pouco mais,
# depois acredito que acontece algum tipo de overfit ou maximo gradiente, por que ele para de performar e 
# fica parado para o A2C e PPO.
# Devido a isso, farei um novo treinamento limitando a 700k steps.

#### Gráficos do tensor Flow

![mean_reward](graficos/2M_1.png)

![mean_ep_length](graficos/2M_2.png)

### Novos treinamentos

In [6]:
import gym
import time
from stable_baselines3 import A2C,PPO,DQN
from stable_baselines3.common.sb2_compat.rmsprop_tf_like import RMSpropTFLike

#carrega o ambiente
env = gym.make('BeamRiderNoFrameskip-v4')
eval_env = gym.make('BeamRiderNoFrameskip-v4')

#instancia o algoritmo de aprendizagem
model = A2C('MlpPolicy', env, verbose=1,create_eval_env=True,ent_coef=0.01,vf_coef=0.25,policy_kwargs=dict(optimizer_class=RMSpropTFLike, optimizer_kwargs=dict(eps=1e-5)),tensorboard_log="/tmp/stable-baselines/BeamRiderNoFrameskip-v4/custon2_a2c")

#treina o algoritmo
model.learn(total_timesteps=700000,eval_env=eval_env,eval_freq=175000, n_eval_episodes=1)

model.save("modelos/a2c/recigio_beamrider_700k")
del model 



Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to /tmp/stable-baselines/BeamRiderNoFrameskip-v4/custon2_a2c\A2C_1
-------------------------------------
| time/                 |           |
|    fps                | 116       |
|    iterations         | 100       |
|    time_elapsed       | 4         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -2.2      |
|    explained_variance | -4.97e+04 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | 0.0732    |
|    value_loss         | 0.0445    |
-------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 143      |
|    iterations         | 20

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 6.09e+03 |
|    ep_rew_mean        | 528      |
| time/                 |          |
|    fps                | 177      |
|    iterations         | 1600     |
|    time_elapsed       | 45       |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -2.2     |
|    explained_variance | -74.5    |
|    learning_rate      | 0.0007   |
|    n_updates          | 1599     |
|    policy_loss        | -0.0153  |
|    value_loss         | 5.67e-05 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 6.09e+03 |
|    ep_rew_mean        | 528      |
| time/                 |          |
|    fps                | 177      |
|    iterations         | 1700     |
|    time_elapsed       | 47       |
|    total_timesteps    | 8500     |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 5.46e+03 |
|    ep_rew_mean        | 374      |
| time/                 |          |
|    fps                | 181      |
|    iterations         | 2900     |
|    time_elapsed       | 79       |
|    total_timesteps    | 14500    |
| train/                |          |
|    entropy_loss       | -2.2     |
|    explained_variance | -21.9    |
|    learning_rate      | 0.0007   |
|    n_updates          | 2899     |
|    policy_loss        | 0.00793  |
|    value_loss         | 1.7e-05  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 5.46e+03 |
|    ep_rew_mean        | 374      |
| time/                 |          |
|    fps                | 181      |
|    iterations         | 3000     |
|    time_elapsed       | 82       |
|    total_timesteps    | 15000    |
| train/                |          |
|

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 5.22e+03  |
|    ep_rew_mean        | 330       |
| time/                 |           |
|    fps                | 181       |
|    iterations         | 4200      |
|    time_elapsed       | 115       |
|    total_timesteps    | 21000     |
| train/                |           |
|    entropy_loss       | -2.2      |
|    explained_variance | -1.01e+03 |
|    learning_rate      | 0.0007    |
|    n_updates          | 4199      |
|    policy_loss        | -0.00253  |
|    value_loss         | 3.53e-06  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 5.22e+03  |
|    ep_rew_mean        | 330       |
| time/                 |           |
|    fps                | 181       |
|    iterations         | 4300      |
|    time_elapsed       | 118       |
|    total_timesteps    | 21500     |
| train/    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 5.22e+03  |
|    ep_rew_mean        | 330       |
| time/                 |           |
|    fps                | 183       |
|    iterations         | 5500      |
|    time_elapsed       | 149       |
|    total_timesteps    | 27500     |
| train/                |           |
|    entropy_loss       | -2.2      |
|    explained_variance | -2.58e+03 |
|    learning_rate      | 0.0007    |
|    n_updates          | 5499      |
|    policy_loss        | 0.00122   |
|    value_loss         | 1.87e-06  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 5.57e+03 |
|    ep_rew_mean        | 343      |
| time/                 |          |
|    fps                | 183      |
|    iterations         | 5600     |
|    time_elapsed       | 152      |
|    total_timesteps    | 28000    |
| train/             

...

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 5.23e+03 |
|    ep_rew_mean        | 349      |
| time/                 |          |
|    fps                | 190      |
|    iterations         | 136200   |
|    time_elapsed       | 3569     |
|    total_timesteps    | 681000   |
| train/                |          |
|    entropy_loss       | -2.2     |
|    explained_variance | -196     |
|    learning_rate      | 0.0007   |
|    n_updates          | 136199   |
|    policy_loss        | -0.00062 |
|    value_loss         | 3.05e-07 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 5.23e+03 |
|    ep_rew_mean        | 349      |
| time/                 |          |
|    fps                | 190      |
|    iterations         | 136300   |
|    time_elapsed       | 3571     |
|    total_timesteps    | 681500   |
| train/                |          |
|

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 5.24e+03  |
|    ep_rew_mean        | 348       |
| time/                 |           |
|    fps                | 190       |
|    iterations         | 137500    |
|    time_elapsed       | 3602      |
|    total_timesteps    | 687500    |
| train/                |           |
|    entropy_loss       | -2.19     |
|    explained_variance | -5.9e+03  |
|    learning_rate      | 0.0007    |
|    n_updates          | 137499    |
|    policy_loss        | -0.000444 |
|    value_loss         | 9.11e-07  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 5.24e+03 |
|    ep_rew_mean        | 348      |
| time/                 |          |
|    fps                | 190      |
|    iterations         | 137600   |
|    time_elapsed       | 3605     |
|    total_timesteps    | 688000   |
| train/             

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 5.26e+03 |
|    ep_rew_mean        | 348      |
| time/                 |          |
|    fps                | 190      |
|    iterations         | 138800   |
|    time_elapsed       | 3636     |
|    total_timesteps    | 694000   |
| train/                |          |
|    entropy_loss       | -2.19    |
|    explained_variance | 0.359    |
|    learning_rate      | 0.0007   |
|    n_updates          | 138799   |
|    policy_loss        | -0.00422 |
|    value_loss         | 5.47e-06 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 5.26e+03 |
|    ep_rew_mean        | 348      |
| time/                 |          |
|    fps                | 190      |
|    iterations         | 138900   |
|    time_elapsed       | 3638     |
|    total_timesteps    | 694500   |
| train/                |          |
|

New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5.26e+03 |
|    ep_rew_mean     | 348      |
| time/              |          |
|    fps             | 189      |
|    iterations      | 140000   |
|    time_elapsed    | 3698     |
|    total_timesteps | 700000   |
---------------------------------


TypeError: __init__() got an unexpected keyword argument 'cliprange'

In [7]:

env.reset()
eval_env.reset()

#instancia o algoritmo de aprendizagem
model = PPO('MlpPolicy', env, verbose=1, vf_coef= 0.5, ent_coef=0.01, create_eval_env=True, batch_size=256,clip_range=0.1,tensorboard_log="/tmp/stable-baselines/BeamRiderNoFrameskip-v4/custon2_ppo")

#treina o algoritmo
model.learn(total_timesteps=700000,eval_env=eval_env,eval_freq=175000, n_eval_episodes=1)

model.save("modelos/ppo/recigio_beamrider_700k")
del model 

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to /tmp/stable-baselines/BeamRiderNoFrameskip-v4/custon2_ppo\PPO_1
-----------------------------
| time/              |      |
|    fps             | 403  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 219         |
|    iterations           | 2           |
|    time_elapsed         | 18          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.002402385 |
|    clip_fraction        | 0.116       |
|    clip_range           | 0.1         |
|    entropy_loss         | -2.2        |

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.03e+03    |
|    ep_rew_mean          | 411         |
| time/                   |             |
|    fps                  | 165         |
|    iterations           | 11          |
|    time_elapsed         | 135         |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.002131322 |
|    clip_fraction        | 0.121       |
|    clip_range           | 0.1         |
|    entropy_loss         | -2.18       |
|    explained_variance   | -0.0422     |
|    learning_rate        | 0.0003      |
|    loss                 | 11.4        |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.00927    |
|    value_loss           | 26.8        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.03

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.42e+03     |
|    ep_rew_mean          | 387          |
| time/                   |              |
|    fps                  | 161          |
|    iterations           | 21           |
|    time_elapsed         | 266          |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 0.0061480897 |
|    clip_fraction        | 0.227        |
|    clip_range           | 0.1          |
|    entropy_loss         | -2.16        |
|    explained_variance   | -5.37        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0396       |
|    n_updates            | 200          |
|    policy_gradient_loss | -0.0114      |
|    value_loss           | 0.355        |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.85e+03     |
|    ep_rew_mean          | 435          |
| time/                   |              |
|    fps                  | 160          |
|    iterations           | 31           |
|    time_elapsed         | 396          |
|    total_timesteps      | 63488        |
| train/                  |              |
|    approx_kl            | 0.0025642086 |
|    clip_fraction        | 0.075        |
|    clip_range           | 0.1          |
|    entropy_loss         | -2.15        |
|    explained_variance   | -0.111       |
|    learning_rate        | 0.0003       |
|    loss                 | 20.7         |
|    n_updates            | 300          |
|    policy_gradient_loss | -0.0107      |
|    value_loss           | 35           |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 6.34e+03   |
|    ep_rew_mean          | 423        |
| time/                   |            |
|    fps                  | 159        |
|    iterations           | 41         |
|    time_elapsed         | 526        |
|    total_timesteps      | 83968      |
| train/                  |            |
|    approx_kl            | 0.00283902 |
|    clip_fraction        | 0.122      |
|    clip_range           | 0.1        |
|    entropy_loss         | -2.13      |
|    explained_variance   | 0.024      |
|    learning_rate        | 0.0003     |
|    loss                 | 13.4       |
|    n_updates            | 400        |
|    policy_gradient_loss | -0.0118    |
|    value_loss           | 31.4       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.34e+03    |
|    ep_rew_m

...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.06e+03     |
|    ep_rew_mean          | 449          |
| time/                   |              |
|    fps                  | 155          |
|    iterations           | 298          |
|    time_elapsed         | 3931         |
|    total_timesteps      | 610304       |
| train/                  |              |
|    approx_kl            | 0.0060197874 |
|    clip_fraction        | 0.145        |
|    clip_range           | 0.1          |
|    entropy_loss         | -1.53        |
|    explained_variance   | -0.0168      |
|    learning_rate        | 0.0003       |
|    loss                 | 8.6          |
|    n_updates            | 2970         |
|    policy_gradient_loss | -0.00557     |
|    value_loss           | 24.2         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.04e+03    |
|    ep_rew_mean          | 453         |
| time/                   |             |
|    fps                  | 155         |
|    iterations           | 308         |
|    time_elapsed         | 4063        |
|    total_timesteps      | 630784      |
| train/                  |             |
|    approx_kl            | 0.005177203 |
|    clip_fraction        | 0.0929      |
|    clip_range           | 0.1         |
|    entropy_loss         | -1.48       |
|    explained_variance   | -5.53e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 20.1        |
|    n_updates            | 3070        |
|    policy_gradient_loss | -0.00318    |
|    value_loss           | 59.1        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.04e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6e+03       |
|    ep_rew_mean          | 456         |
| time/                   |             |
|    fps                  | 155         |
|    iterations           | 318         |
|    time_elapsed         | 4193        |
|    total_timesteps      | 651264      |
| train/                  |             |
|    approx_kl            | 0.007301066 |
|    clip_fraction        | 0.158       |
|    clip_range           | 0.1         |
|    entropy_loss         | -1.44       |
|    explained_variance   | -0.0023     |
|    learning_rate        | 0.0003      |
|    loss                 | 18.7        |
|    n_updates            | 3170        |
|    policy_gradient_loss | -0.00635    |
|    value_loss           | 25.2        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6e+03 

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.01e+03    |
|    ep_rew_mean          | 459         |
| time/                   |             |
|    fps                  | 155         |
|    iterations           | 328         |
|    time_elapsed         | 4323        |
|    total_timesteps      | 671744      |
| train/                  |             |
|    approx_kl            | 0.005020452 |
|    clip_fraction        | 0.173       |
|    clip_range           | 0.1         |
|    entropy_loss         | -1.37       |
|    explained_variance   | 0.00549     |
|    learning_rate        | 0.0003      |
|    loss                 | 15.4        |
|    n_updates            | 3270        |
|    policy_gradient_loss | -0.00376    |
|    value_loss           | 36.7        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.03

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.03e+03     |
|    ep_rew_mean          | 463          |
| time/                   |              |
|    fps                  | 155          |
|    iterations           | 338          |
|    time_elapsed         | 4454         |
|    total_timesteps      | 692224       |
| train/                  |              |
|    approx_kl            | 0.0073981676 |
|    clip_fraction        | 0.205        |
|    clip_range           | 0.1          |
|    entropy_loss         | -1.32        |
|    explained_variance   | -0.000654    |
|    learning_rate        | 0.0003       |
|    loss                 | 4.68         |
|    n_updates            | 3370         |
|    policy_gradient_loss | -0.00306     |
|    value_loss           | 8            |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

In [9]:

env.reset()
eval_env.reset()

#instancia o algoritmo de aprendizagem
model = DQN('MlpPolicy', env, verbose=1,batch_size=128, buffer_size=10000,gradient_steps=1,exploration_fraction=0.1,create_eval_env=True,tensorboard_log="/tmp/stable-baselines/BeamRiderNoFrameskip-v4/custon2_dqn")

#treina o algoritmo
model.learn(total_timesteps=700000,eval_env=eval_env,eval_freq=175000, n_eval_episodes=1)

model.save("modelos/qdn/recigio_beamrider_700k")
del model 

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to /tmp/stable-baselines/BeamRiderNoFrameskip-v4/custon2_dqn\DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.71e+03 |
|    ep_rew_mean      | 363      |
|    exploration rate | 0.69     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1021     |
|    time_elapsed     | 22       |
|    total timesteps  | 22823    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.38e+03 |
|    ep_rew_mean      | 396      |
|    exploration rate | 0.416    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1026     |
|    t

New best mean reward!
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.89e+03 |
|    ep_rew_mean      | 401      |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 60       |
|    fps              | 114      |
|    time_elapsed     | 3095     |
|    total timesteps  | 353182   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.34     |
|    n_updates        | 75795    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.84e+03 |
|    ep_rew_mean      | 396      |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 64       |
|    fps              | 113      |
|    time_elapsed     | 3293     |
|    total timesteps  | 373828   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000146 

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.48e+03 |
|    ep_rew_mean      | 360      |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 120      |
|    fps              | 107      |
|    time_elapsed     | 6180     |
|    total timesteps  | 663616   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000291 |
|    n_updates        | 153403   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.45e+03 |
|    ep_rew_mean      | 353      |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 124      |
|    fps              | 107      |
|    time_elapsed     | 6359     |
|    total timesteps  | 681625   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.336    |
|    n_updates      

c:\xampp\htdocs\posfurb\aprendizadodemaquina\python\.env\lib\site-packages\stable_baselines3\common\save_util.py:276: UserWarning: Path 'modelos\qdn' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


#### Testando novos resultados

In [13]:
import gym
import time
from stable_baselines3 import A2C,PPO,DQN
from stable_baselines3.common.sb2_compat.rmsprop_tf_like import RMSpropTFLike

#carrega o ambiente
env = gym.make('BeamRiderNoFrameskip-v4')

model = A2C.load("modelos/a2c/recigio_beamrider_700k", env=env)

# importando biblioteca de avaliação e avaliando por 10 episodios
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10, render=True)

print(mean_reward)
print(std_reward)


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
756.0
0.0


In [ ]:
# O algoritmo achou um lugar que considerou mais ideal para se proteger dos tiros e atingir alguns inimigos
# ao invés de se movimenntar

In [1]:
import gym
import time
from stable_baselines3 import A2C,PPO,DQN
from stable_baselines3.common.sb2_compat.rmsprop_tf_like import RMSpropTFLike

#carrega o ambiente
env = gym.make('BeamRiderNoFrameskip-v4')

model = PPO.load("modelos/ppo/recigio_beamrider_700k", env=env)

# importando biblioteca de avaliação e avaliando por 10 episodios
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10, render=True)

print(mean_reward)
print(std_reward)


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


c:\xampp\htdocs\posfurb\aprendizadodemaquina\python\.env\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Não é possível alterar o modo de thread depois de o mesmo estar definido
  warnings.warn(str(err))


176.0
0.0


In [ ]:
# tentou achar um padrão de movimentacao que fizesse mais pontos, sem realmente observar os inimigos

In [2]:
import gym
import time
from stable_baselines3 import A2C,PPO,DQN
from stable_baselines3.common.sb2_compat.rmsprop_tf_like import RMSpropTFLike

#carrega o ambiente
env = gym.make('BeamRiderNoFrameskip-v4')

model = DQN.load("modelos/qdn/recigio_beamrider_700k", env=env)

# importando biblioteca de avaliação e avaliando por 10 episodios
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10, render=True)

print(mean_reward)
print(std_reward)


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
39.6
60.4899991734171


In [ ]:
#um resultado um pouco melhor que o ppo

### Resultados

In [26]:
# Os resultados com 700k se mostraram um pouco melhores. Conteudo, para acalnsar resultados melhores, também,
# foram atualizados alguns parametros parametrizados. Me parece que esse cenário em especifico precisa de um.
# tunning melhor. Vendo agora, talvez o problema não seja exatamente overfit, mas parametrização.
# Também, me parece que seria necessário bem mais etapas de treinamento.

![mean_reward](graficos/700k_1.png)

![mean_reward](graficos/700k_2.png)

### Treinando utilizando modelos RL Baselines3 Zoooo

In [ ]:
# Devido a algum bug, apagou os relatorios de execução dos treinamentos.

In [14]:
%cd rl-baselines3-zoo/

C:\xampp\htdocs\posfurb\reinforcementlearning\rl-baselines3-zoo


#### A2C - Treinamento com 2M de steps

In [ ]:
!python train.py --algo a2c --env BeamRiderNoFrameskip-v4 --tensorboard-log /tmp/stable-baselines/ -n 2000000 --save-freq 200000 --eval-freq 200000 --eval-episodes 10 --n-eval-envs 10

#### PPO - Treinamento com 2M de steps

In [ ]:
#devido a um bug ele escreveu metade em cada celula
!python train.py --algo ppo --env BeamRiderNoFrameskip-v4 --tensorboard-log /tmp/stable-baselines/ -n 2000000 --save-freq 200000 --eval-freq 200000 --eval-episodes 10 --n-eval-envs 10

#### DQN - Treinamento com 2M de steps

In [ ]:
!python train.py --algo dqn --env BeamRiderNoFrameskip-v4 --tensorboard-log /tmp/stable-baselines/ -n 2000000 --save-freq 200000 --eval-freq 200000 --eval-episodes 10 --n-eval-envs 10

#### Visualização dos resultados - Utilizando a biblioteca enjoy apontando pra pasta de saida dos treinamentos

In [ ]:
!python enjoy.py --algo a2c --env BeamRiderNoFrameskip-v4 --folder logs/ -n 5000

In [ ]:
!python enjoy.py --algo ppo --env BeamRiderNoFrameskip-v4 --folder logs/ -n 5000

In [ ]:
!python enjoy.py --algo dqn --env BeamRiderNoFrameskip-v4 --folder logs/ -n 5000

In [ ]:
# Os resultados foram bons, não ótimos, a ia consegue algum sucesso. Parece um pouco aleatorio os movimentos,
# acredito que devido a complexidade, precisava de mais treinamento. 
# O agente pronto da biblioteca, performa bem melhor.

![mean_reward ](graficos/Zoo_11.png)

![mean_reward](graficos/Zoo_11.png)

![mean_reward](graficos/Zoo_22.png)

# Resultados

## Tabela comparativa

In [28]:
import pandas as pd
data = [
        #analise
        ['A2C', '2M',0,0], 
        ['A2C', '700k',756,0], 
        ['A2C ZOO', '700k',578,'NA'], 
        ['PPO', '2M',0,0], 
        ['PPO', '700k',176,0], 
        ['PPO ZOO', '700k',649,'NA'],
        ['DQN', '2M',44,0], 
        ['DQN', '700k',39,60], 
        ['DQN ZOO', '700k',3432,'NA']
        # modelo 1
       ]
  
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Algoritmo', 'TimeSteps','mean_reward','std_reward'])
  
# print dataframe.
df.sort_values(by=['mean_reward'], ascending=False)

,Algoritmo,TimeSteps,mean_reward,std_reward
8,DQN ZOO,700k,3432,NA
1,A2C,700k,756,0
5,PPO ZOO,700k,649,NA
2,A2C ZOO,700k,578,NA
4,PPO,700k,176,0
6,DQN,2M,44,0
7,DQN,700k,39,60
0,A2C,2M,0,0
3,PPO,2M,0,0


In [30]:
# Comparativamente os resultados do zoo parecem muito melhor tunados, numa mesma quantidade de timesptep.
# Talvez seja alguma parametrização ou configuração minha que falhou, mas acredito que não.
# Os algoritmos que eu treinei, tentavam encontrar um ponto otimo no cenário para ficarem parados, enquanto
# os algoritmos do zoom tentam se mover observando os inimigos. Acredito que que meus algoritmos estão empacando
# em algum maximo gradiente ou treinando com poucas etapas. Com precisei fazer vários experimentos, não
# tive tempo de treinar com mais de 2M de timesteps os modelos. Demoram em média 3 ou 4 horas cada um.
# Também, me parece que o DQN, baseado em Q-Learning parece o algoritmo mais promissor. 

## Melhores resultados A2C e DQN ZOO - Videos

In [6]:
import gym
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
from stable_baselines3 import A2C,PPO,DQN

#carrega o ambiente
env_model = gym.make('BeamRiderNoFrameskip-v4')
model = A2C.load("modelos/a2c/recigio_beamrider_700k", env=env_model)

env_id = 'BeamRiderNoFrameskip-v4'
video_folder = 'videos/'
video_length = 3500

env = DummyVecEnv([lambda: gym.make(env_id)])
obs = env.reset()

# Record the video starting at the first step
env = VecVideoRecorder(env, video_folder,
                       record_video_trigger=lambda x: x == 0, video_length=video_length,
                       name_prefix="random-agent-{}".format(env_id))

env.reset()
for _ in range(video_length + 1):
  action, _states = model.predict(obs, deterministic=True)
  obs, rewards, done, info = env.step(action)
# Save the video
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Saving video to C:\xampp\htdocs\posfurb\reinforcementlearning\videos\random-agent-BeamRiderNoFrameskip-v4-step-0-to-step-3500.mp4


In [1]:
%cd rl-baselines3-zoo/

C:\xampp\htdocs\posfurb\reinforcementlearning\rl-baselines3-zoo


In [2]:
!python -m utils.record_video --algo dqn --env BeamRiderNoFrameskip-v4 --folder logs/ -n 3500

Loading latest experiment, id=1
Stacking 4 frames
Wrapping the env in a VecTransposeImage.
Saving video to C:\xampp\htdocs\posfurb\reinforcementlearning\rl-baselines3-zoo\logs\dqn\BeamRiderNoFrameskip-v4_1\videos\final-model-dqn-BeamRiderNoFrameskip-v4-step-0-to-step-3500.mp4


C:\xampp\htdocs\posfurb\aprendizadodemaquina\python\.env\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Não é possível alterar o modo de thread depois de o mesmo estar definido
  warnings.warn(str(err))
Exception ignored in: <function VecVideoRecorder.__del__ at 0x00000234054FC5E0>
Traceback (most recent call last):
  File "C:\xampp\htdocs\posfurb\aprendizadodemaquina\python\.env\lib\site-packages\stable_baselines3\common\vec_env\vec_video_recorder.py", line 113, in __del__
  File "C:\xampp\htdocs\posfurb\aprendizadodemaquina\python\.env\lib\site-packages\stable_baselines3\common\vec_env\vec_video_recorder.py", line 109, in close
AttributeError: 'NoneType' object has no attribute 'close'


In [3]:
from IPython.display import Video
%cd ..

C:\xampp\htdocs\posfurb\reinforcementlearning


### DQN ZOO Video

In [5]:
Video("videos/final-model-dqn-BeamRiderNoFrameskip-v4-step-0-to-step-3500.mp4")

### A2C Video

In [7]:
Video("videos/random-agent-BeamRiderNoFrameskip-v4-step-0-to-step-3500.mp4")